jan+(11*22) -------2012
2013-2024 12*12*2
2025 jan-oct   10*2
total is 331


In [42]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE_URL = "https://www.fpi.nsdl.co.in/web/Reports/FPI_Fortnightly_Selection.aspx"
ROOT = "https://www.fpi.nsdl.co.in/web/"

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "https://www.google.com/",
    "Connection": "keep-alive"
}

response = requests.get(BASE_URL, headers=headers, timeout=10)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

select_box = soup.find("select", {"id": "ddlfortnighly"})
options = select_box.find_all("option")

urls = []

for opt in options:
    raw = opt.get("value")
    full_url = urljoin(ROOT, raw.replace("~/", ""))
    urls.append(full_url)

for u in urls:
    print(u)


https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_Oct312025.html
https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_Oct152025.html
https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_Sep302025.html
https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_Sep152025.html
https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_Aug312025.html
https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_Aug152025.html
https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_July312025.html
https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_July152025.html
https://www.fpi.nsdl.co.in/web/StaticR

In [ ]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

def fetch_table(url):
    options = webdriver.ChromeOptions()

    # Important: run in NON-HEADLESS mode (NSDL blocks headless)
    # options.add_argument("--headless")  <-- DO NOT ENABLE THIS

    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

    # Apply stealth (this is IMPORTANT)
    stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
    )

    driver.get(url)
    time.sleep(4)   # allow JS + table to load fully

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "lxml")
    table = soup.find("table")

    if table is None:
        raise Exception("No table found — NSDL still blocking. Try increasing wait to 7 sec.")

    df = pd.read_html(str(table))[0]
    
    path = "/home/punitneh/Desktop/all_data/fpi/nsdl_csv"
    name = url.split("/")[-1].replace(".html", "").replace("FIIInvestSector_", "")
    df.to_csv(f"{path}/{name}.csv", index=False)
    print("Saved table!")


# for ur in urls:
#     df = fetch_table(ur)
#     name = ur.split("/")[-1].replace(".html", "").replace("FIIInvestSector_", "")
#     df.to_csv(f"{path}/{name}.csv", index=False)
#     print("Saved table!")



In [ ]:
import cloudscraper
import pandas as pd
from bs4 import BeautifulSoup
import os
pathS = "/home/punitneh/Desktop/deloitte/fpi/nsdl_csv"
os.makedirs(pathS, exist_ok=True) 
def fetch_table_to_csv(url):

    scraper = cloudscraper.create_scraper(
        browser={
            "browser": "chrome",
            "platform": "linux",
            "mobile": False
        }
    )

    print("Fetching:", url)
    html = scraper.get(url).text

    soup = BeautifulSoup(html, "lxml")
    table = soup.find("table")

    if table is None:
        print("❌ No table found FROM FIRST METHOD")
        fetch_table(url)
        return

    df = pd.read_html(str(table))[0]
    df = df.dropna(how="all").dropna(axis=1, how="all")

    filename = url.split("/")[-1].replace(".html", ".csv")
    filename = filename.replace("FIIInvestSector_", "")
    path = os.path.join(pathS, filename)
    df.to_csv(path, index=False)

    print("✅ Saved:", path)


for url in urls:
    fetch_table_to_csv(url) 




In [ ]:
import os

from pandas import array

folder = "/home/punitneh/Desktop/deloitte/fpi/nsdl_csv"
map = {}
file = os.listdir(folder)
for f in file:
    year = f.split("_")[0]
    if year not in map:
        map[year] = []
    map[year].append(f)

for year in map:
    print("Processing year:", year , "with files:", len(map[year]))   staggered hit



Processing year: August312019.csv with files: 1
Processing year: July152022.csv with files: 1
Processing year: Sep302020.csv with files: 1
Processing year: June302012.csv with files: 1
Processing year: Mar312022.csv with files: 1
Processing year: FIIInestSector with files: 24
Processing year: July152021.csv with files: 1
Processing year: March312020.csv with files: 1
Processing year: April152020.csv with files: 1
Processing year: Feb152018.csv with files: 1
Processing year: Oct312025.csv with files: 1
Processing year: July312025.csv with files: 1
Processing year: Feb282025.csv with files: 1
Processing year: Aug152025.csv with files: 1
Processing year: Nov302022.csv with files: 1
Processing year: Sep152024.csv with files: 1
Processing year: Nov302016.csv with files: 1
Processing year: July152018.csv with files: 1
Processing year: Sep152025.csv with files: 1
Processing year: Mar312025.csv with files: 1
Processing year: May312024.csv with files: 1
Processing year: June302019.csv with file

In [52]:
len(file)

331